## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [557]:
# %load functions.py
import functions

## Step 2: Compute the camera calibration using chessboard images

In [107]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space (real undistorted chessboard corners)
               # These are the known object coordinates of the chessboard corners
               # x, y, and z from the top left corner (0,0,0) to (x, y, 0) --> flat 
imgpoints = [] # 2d points in image plane. (coordinates of the corners)

# Make a list of calibration images
images = glob.glob('1_camera_cal/calibration*.jpg')

# Array to store valid_imgages
valid_imgs = []

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        valid_imgs.append(fname)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
cv2.destroyAllWindows()

## Step 3: Correct for Distortion (and save undistorted images)

In [108]:
import re
undist_imgs = []
for i in range(0, len(valid_imgs)):
    img = cv2.imread(valid_imgs[i])
    img_num = ''.join(re.findall("\d",(valid_imgs[i])))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    #cv2.imshow('img',undist)
    #cv2.waitKey(500)
    undist_path = str('2_camera_undist'+'/undistort'+img_num+'.jpg')
    undist_imgs.append(undist_path)
    cv2.imwrite(undist_path, undist)
cv2.destroyAllWindows()

## Step 4: Perspective Transform (and save transformed images)

In [109]:
nx = 9
ny = 6
trans_paths = []
for i in range(0, len(undist_imgs)):
    img = cv2.imread(undist_imgs[i])
    img_num = ''.join(re.findall("\d",(undist_imgs[i]))) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    if ret == True:
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        offset = 100
        img_size = (gray.shape[1], gray.shape[0])
        src = np.float32([corners[0], corners[nx-1],corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset],
                          [img_size[0]-offset, img_size[1]-offset],
                          [offset, img_size[1]-offset]])
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(img, M, img_size)
        cv2.imshow('img',warped)
        cv2.waitKey(500)
        trans_path = str('3_camera_transform'+'/transform'+img_num+'.jpg')
        trans_paths.append(trans_path)
        cv2.imwrite(trans_path, warped)
cv2.destroyAllWindows()

## Step 5: Undistort camera images of road

In [276]:
images = glob.glob('4_test_images/*.jpg')
undist_imgs = []
for fname in images:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    #cv2.imshow('img',undist)
    #cv2.waitKey(500)
    undist_path = str('5_test_images_undist' + '/' + img_name +'_undist.jpg')
    undist_imgs.append(undist_path)
    cv2.imwrite(undist_path, undist)
cv2.destroyAllWindows()

## Step 6a: Applying Sobel X Threshold

In [552]:
sobelx_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    gradx = 255*abs_sobel_thresh(img, orient = 'x', thresh_min = 20, thresh_max = 100)
    #plt.imshow(grad_binary, cmap='gray')
    cv2.imshow('img',gradx)
    cv2.waitKey(500)
    sobelx_path = str('6A_test_images_sobelx' + '/' + img_name +'_sobelx.jpg')
    sobelx_imgs.append(sobelx_path)
    cv2.imwrite(sobelx_path, gradx)
cv2.destroyAllWindows()

## Step 6B: Applying Sobel Y Threshold

In [8]:
sobely_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    grady = 255*abs_sobel_thresh(img, orient = 'y', thresh_min = 50, thresh_max = 100)
    #plt.imshow(grad_binary, cmap='gray')
    cv2.imshow('img',grady)
    cv2.waitKey(500)
    sobely_path = str('6B_test_images_sobely' + '/' + img_name +'_sobely.jpg')
    sobely_imgs.append(sobely_path)
    cv2.imwrite(sobely_path, grady)
cv2.destroyAllWindows()

## Step 6C: Applying sobel x and y magnitude threshold

In [10]:
sobelxy_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    mag_binary = 255 * mag_thresh(img, sobel_kernel=3, thresh_min = 30, thresh_max = 100)
    #plt.imshow(grad_binary, cmap='gray')
    cv2.imshow('img',mag_binary)
    cv2.waitKey(500)
    sobelxy_path = str('6C_test_images_sobelxy' + '/' + img_name +'_sobelxy.jpg')
    sobelxy_imgs.append(sobelxy_path)
    cv2.imwrite(sobelxy_path, mag_binary)
cv2.destroyAllWindows()

## Step 6D: Direction of Gradient Threshold

In [11]:
gradient_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    dir_binary = 255 * dir_threshold(img, sobel_kernel=15, thresh = (0.7,1.3))
    cv2.imshow('img',dir_binary)
    cv2.waitKey(500)
    gradient_path = str('6D_test_images_gradient' + '/' + img_name +'_gradient.jpg')
    gradient_imgs.append(gradient_path)
    cv2.imwrite(gradient_path, dir_binary)
cv2.destroyAllWindows()

## Step 6E: Combining the gradient threshold (X,Y, magnitude, and direction)

In [354]:
combined_thresh_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    gradx = 255*abs_sobel_thresh(img, orient = 'x', thresh_min = 20, thresh_max = 100)
    grady = 255*abs_sobel_thresh(img, orient = 'y', thresh_min = 50, thresh_max = 100)
    mag_binary = 255 * mag_thresh(img, sobel_kernel=3, thresh_min = 30, thresh_max = 100)
    dir_binary = 255 * dir_threshold(img, sobel_kernel=15, thresh = (0.7,1.3))
    
    combined = np.zeros_like(gradx)
    combined[((gradx == 255) & (grady == 255)) | ((mag_binary == 1) & (dir_binary == 1))] = 255
    cv2.imshow('img',combined)
    cv2.waitKey(500)
    combined_thresh_path = str('6E_test_images_combinedthr' + '/' + img_name +'_combinedthr.jpg')
    combined_thresh_imgs.append(combined_thresh_path)
    cv2.imwrite(combined_thresh_path, combined)
cv2.destroyAllWindows()

## Step 6F: Applying Color Threshold (HLS)

In [350]:
# Define a function that thresholds the S-channel of HLS
def hls_select(img, thresh_h=(0,179), thresh_s = (0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    H = hls[:,:,0] 
    L = hls[:,:,1]
    S = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_output[((S > thresh_s[0]) & (S <= thresh_s[1])) & ((H > thresh_h[0]) & (H <= thresh_h[1]))] = 1
    return binary_output

hls_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    hls_binary = 255 * hls_select(img, thresh_h = (15, 100), thresh_s = (90, 255))
    cv2.imshow('img',hls_binary)
    cv2.waitKey(500)
    hls_path = str('6F_test_images_hls' + '/' + img_name +'_hls.jpg')
    hls_imgs.append(hls_path)
    cv2.imwrite(hls_path, hls_binary)
cv2.destroyAllWindows()

## Step 6G: Combine Gradient and Color Threshold

In [403]:
combined_all_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    
    gradx = 255*abs_sobel_thresh(img, orient = 'x', thresh_min = 20, thresh_max = 100)
    grady = 255*abs_sobel_thresh(img, orient = 'y', thresh_min = 50, thresh_max = 100)
    mag_binary = 255 * mag_thresh(img, sobel_kernel=3, thresh_min = 30, thresh_max = 100)
    dir_binary = 255 * dir_threshold(img, sobel_kernel=15, thresh = (0.7,1.3))
    combined_grad = np.zeros_like(gradx)
    combined_grad[((gradx == 255) & (grady == 255)) | ((mag_binary == 1) & (dir_binary == 1))] = 255
    hls_binary = 255 * hls_select(img, thresh_h = (15, 100),thresh_s = (90, 255))
    combined_all = np.zeros_like(hls_binary)
    combined_all[(hls_binary == 255) | (combined_grad == 255)] = 255
    cv2.imshow('img',combined_all)
    cv2.waitKey(500)
    combined_all_path = str('6G_test_images_combinedall' + '/' + img_name +'_combinedall.jpg')
    combined_all_imgs.append(combined_all_path)
    cv2.imwrite(combined_all_path, combined_all)
cv2.destroyAllWindows()

## Step 7A: Region of interest of lanes

In [360]:
bot_left = (190,imshape[0])
top_left = (600,445)
top_right = (680, 445)
bot_right = (1120,imshape[0])

roi_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    cv2.line(img,bot_left, top_left, (0, 0, 255), 2) 
    cv2.line(img,top_left, top_right, (0, 0, 255), 2)   
    cv2.line(img,top_right, bot_right, (0, 0, 255), 2) 
    cv2.line(img,bot_right, bot_left, (0, 0, 255), 2)  
    
    cv2.imshow('img',img)
    cv2.waitKey(500)
    roi_path = str('7A_test_images_roi' + '/' + img_name +'_roi.jpg')
    roi_imgs.append(roi_path)
    cv2.imwrite(roi_path, img)
cv2.destroyAllWindows()

## Step 7B: ROI Warped

In [361]:
dest_bot_left = (300,imshape[0])
dest_top_left = (300,0)
dest_top_right = (1240-300,0)
dest_bot_right = (1240-300,imshape[0])

roiwarped_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    imshape = img.shape
    src = np.float32([[bot_left,top_left,top_right,bot_right]])
    dst = np.float32([[dest_bot_left, dest_top_left, dest_top_right, dest_bot_right]])  

    M = cv2.getPerspectiveTransform(src, dst)
    roiwarp = cv2.warpPerspective(img, M, (imshape[1], imshape[0]))
    
    cv2.line(roiwarp,dest_bot_left, dest_top_left, (0, 0, 255), 2) 
    cv2.line(roiwarp,dest_top_right, dest_bot_right, (0, 0, 255), 2) 
    
    cv2.imshow('img',roiwarp)
    cv2.waitKey(500)
    roiwarp_path = str('7B_test_images_roi_warp' + '/' + img_name +'_warp.jpg')
    roiwarped_imgs.append(roiwarp_path)
    cv2.imwrite(roiwarp_path, roiwarp)
cv2.destroyAllWindows()

## Step 8: Perspective Transform (Warp the Image)

In [362]:
warp_imgs = []
for fname in combined_all_imgs:
    img = cv2.imread(fname, 0)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    imshape = img.shape
    src = np.float32([[bot_left,top_left,top_right,bot_right]])
    dst = np.float32([[dest_bot_left, dest_top_left, dest_top_right, dest_bot_right]])  
    M = cv2.getPerspectiveTransform(src, dst)
    warp = cv2.warpPerspective(img, M, (imshape[1], imshape[0]))
    cv2.imshow('img',warp)
    cv2.waitKey(500)
    warp_path = str('8_test_images_warp' + '/' + img_name +'_warp.jpg')
    warp_imgs.append(warp_path)
    cv2.imwrite(warp_path, warp)
cv2.destroyAllWindows()

## Step 9: Finding the Lines Sliding Window (Histogram Method) and the Curvature

In [474]:
# Define conversions in x and y from pixels space to meters
#ym_per_pix = 30/720 # meters per pixel in y dimension
#xm_per_pix = 3.7/700 # meters per pixel in x dimension
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/640 # meters per pixel in x dimension
    

hist_imgs = []
for fname in warp_imgs:
    img = cv2.imread(fname, 0)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    out_img, left_fit, right_fit, left_curve, right_curve = fit_polynomial(img)
    cv2.imshow('img',out_img)
    cv2.waitKey(500)
    hist_path = str('9_test_images_hist' + '/' + img_name +'_hist.jpg')
    hist_imgs.append(hist_path)
    cv2.imwrite(hist_path, out_img)
cv2.destroyAllWindows()


## Step 10: Final Output (Image, Lane with color, Curvature text)

In [479]:
final_imgs = []
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/640 # meters per pixel in x dimension
    
for i in range(0,len(undist_imgs)):
    # Read the warped binary warped image
    binary_warped = cv2.imread(warp_imgs[i],0)
    
    # Find the curvature 
    out_img, left_fit, right_fit, left_curve, right_curve = fit_polynomial(binary_warped)

    # Find the region of lane identified 
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]),(0,255,0))
    
    # Find the vehicle offset
    img_mid = (dest_bot_left[0] + dest_bot_right[0]) // 2
    real_mid = (left_fitx[0] + right_fitx[0]) // 2
    offset = (real_mid - img_mid) * xm_per_pix
    
    # Unwarp the warped image to overlay lane lines on top of actual image
    src = np.float32([[bot_left,top_left,top_right,bot_right]])
    dst = np.float32([[dest_bot_left, dest_top_left, dest_top_right, dest_bot_right]])  
    m = cv2.getPerspectiveTransform(dst, src)
    unwarp_binarywarp = cv2.warpPerspective(binary_warped, m, (imshape[1], imshape[0]))
    unwarp_colorwarp  = cv2.warpPerspective(color_warp, m, (imshape[1], imshape[0]))
    
    # Read the undistorted image and overlay lane
    img = cv2.imread(undist_imgs[i])
    img_name = ''.join(re.findall("(?<=/)\w+",undist_imgs[i])) # Find .jpg name after a '/'
    img[unwarp_binarywarp == 255] = (0,0,255)
    result = cv2.addWeighted(img, 1, unwarp_colorwarp, 0.3, 0)
    
    # Put text of curvature radiust and vehicle offset
    cv2.putText(result,"left radius = " + str(int(left_curve)) + " m",(100,500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),3)
    cv2.putText(result,"right radius = " + str(int(right_curve)) + " m",(800, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),3)
    cv2.putText(result, "vehicle offset = " + str(round(offset,2)) + " m", (550, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),3)
    cv2.imshow('img',result)
    cv2.waitKey(500)
    final_path = str('10_test_images_final' + '/' + img_name +'_combinedall.jpg')
    final_imgs.append(final_path)
    cv2.imwrite(final_path, result)
cv2.destroyAllWindows()

## Step 11: Apply to Video Clips

In [540]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
    
def advancedLaneLines_vid(image):
    # Undistort the images
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, image.shape[1:], None, None)
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    # Sobel X
    gradx = 255*abs_sobel_thresh(undist, orient = 'x', thresh_min = 20, thresh_max = 100)
    # Sobel Y
    grady = 255*abs_sobel_thresh(undist, orient = 'y', thresh_min = 50, thresh_max = 100)
    # Sobel Magnitude
    mag_binary = 255 * mag_thresh(undist, sobel_kernel=3, thresh_min = 30, thresh_max = 100)
    # Sobel in direction
    dir_binary = 255 * dir_threshold(undist, sobel_kernel=15, thresh = (0.7,1.3))
    # Combine the Sobel
    combined_grad = np.zeros_like(gradx)
    combined_grad[((gradx == 255) & (grady == 255)) | ((mag_binary == 1) & (dir_binary == 1))] = 255
    # HLS Color
    hls_binary = 255 * hls_select(undist, thresh_h = (15, 100),thresh_s = (90, 255))
    # Combine Everything
    combined_all = np.zeros_like(hls_binary)
    combined_all[(hls_binary == 255) | (combined_grad == 255)] = 255
    # Warp the image
    imshape = combined_all.shape
    src = np.float32([[bot_left,top_left,top_right,bot_right]])
    dst = np.float32([[dest_bot_left, dest_top_left, dest_top_right, dest_bot_right]])  
    M = cv2.getPerspectiveTransform(src, dst)
    warp = cv2.warpPerspective(combined_all, M, (imshape[1], imshape[0]))
    
    # Find the curvature 
    out_img, left_fit, right_fit, left_curve, right_curve = fit_polynomial(warp)
    
    # Find left_fit, right_fit the pixels in the two lines
    left_fit, right_fit = search_around_poly(warp)
    # Find the region of lane identified 
    ploty = np.linspace(0, warp.shape[0]-1, warp.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    warp_zero = np.zeros_like(warp).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]),(0,255,0))
    
    # Find the vehicle offset
    img_mid = (dest_bot_left[0] + dest_bot_right[0]) // 2
    real_mid = (left_fitx[0] + right_fitx[0]) // 2
    offset = (real_mid - img_mid) * xm_per_pix
    
    # Unwarp the warped image to overlay lane lines on top of actual image
    src = np.float32([[bot_left,top_left,top_right,bot_right]])
    dst = np.float32([[dest_bot_left, dest_top_left, dest_top_right, dest_bot_right]])  
    m = cv2.getPerspectiveTransform(dst, src)
    unwarp_binarywarp = cv2.warpPerspective(warp, m, (imshape[1], imshape[0]))
    unwarp_colorwarp  = cv2.warpPerspective(color_warp, m, (imshape[1], imshape[0]))
    
    # Read the undistorted image and overlay lane
    undist[unwarp_binarywarp == 255] = (0,0,255)
    result = cv2.addWeighted(undist, 1, unwarp_colorwarp, 0.3, 0)
    
    # Put text of curvature radiust and vehicle offset
    cv2.putText(result,"left radius = " + str(int(left_curve)) + " m",(100,500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),3)
    cv2.putText(result,"right radius = " + str(int(right_curve)) + " m",(800, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),3)
    cv2.putText(result, "vehicle offset = " + str(round(offset,2)) + " m", (550, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),3)  
    
    # Result
    temp.left_fit = left_fit
    temp.right_fit = right_fit
    return result

challenge_output = '11_video_output/project_video1.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip = VideoFileClip('project_video.mp4').subclip(0,2)
#clip = VideoFileClip('project_video.mp4')
init_img = clip.get_frame(0)
left_fit, right_fit = find_initial_lane_poly(init_img)
temp = userdata()
temp.left_fit = left_fit
temp.right_fit = right_fit
challenge_clip = clip.fl_image(advancedLaneLines_vid)
%time challenge_clip.write_videofile(challenge_output, audio=False)

KeyboardInterrupt: 

In [509]:
result = cv2.addWeighted(undist, 1, unwarp_colorwarp, 0.3, 0)


In [515]:
cv2.imshow('img',cv2.imread(undist_imgs[0]))

In [542]:
result = advancedLaneLines_vid(cv2.imread(undist_imgs[0]))
cv2.imshow('img',result)

In [541]:
left_fit, right_fit = find_initial_lane_poly(cv2.imread(undist_imgs[0]))
temp.left_fit = left_fit
temp.right_fit = right_fit
print(left_fit)
print(right_fit)

[ 2.61789241e-04 -5.16106342e-01  5.10061303e+02]
[ 5.16878554e-04 -7.90554111e-01  1.25321568e+03]


In [534]:
image = (cv2.imread(undist_imgs[0]))
# Undistort the images
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, image.shape[1:], None, None)
undist = cv2.undistort(image, mtx, dist, None, mtx)
# Sobel X
gradx = 255*abs_sobel_thresh(undist, orient = 'x', thresh_min = 20, thresh_max = 100)
# Sobel Y
grady = 255*abs_sobel_thresh(undist, orient = 'y', thresh_min = 50, thresh_max = 100)
# Sobel Magnitude
mag_binary = 255 * mag_thresh(undist, sobel_kernel=3, thresh_min = 30, thresh_max = 100)
# Sobel in direction
dir_binary = 255 * dir_threshold(undist, sobel_kernel=15, thresh = (0.7,1.3))
# Combine the Sobel
combined_grad = np.zeros_like(gradx)
combined_grad[((gradx == 255) & (grady == 255)) | ((mag_binary == 1) & (dir_binary == 1))] = 255
# HLS Color
hls_binary = 255 * hls_select(undist, thresh_h = (15, 100),thresh_s = (90, 255))
# Combine Everything
combined_all = np.zeros_like(hls_binary)
combined_all[(hls_binary == 255) | (combined_grad == 255)] = 255
# Warp the image
imshape = combined_all.shape
src = np.float32([[bot_left,top_left,top_right,bot_right]])
dst = np.float32([[dest_bot_left, dest_top_left, dest_top_right, dest_bot_right]])  
M = cv2.getPerspectiveTransform(src, dst)
warp = cv2.warpPerspective(combined_all, M, (imshape[1], imshape[0]))

# Find the curvature 
out_img, left_fit, right_fit, left_curve, right_curve = fit_polynomial(warp)

# Find left_fit, right_fit the pixels in the two lines
left_fit, right_fit = search_around_poly(warp)

# Find the region of lane identified 
ploty = np.linspace(0, warp.shape[0]-1, warp.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
warp_zero = np.zeros_like(warp).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
pts_left = np.array([np.transpose(np.vstack([left_fitx,ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))
cv2.fillPoly(color_warp, np.int_([pts]),(0,255,0))

# Find the vehicle offset
img_mid = (dest_bot_left[0] + dest_bot_right[0]) // 2
real_mid = (left_fitx[0] + right_fitx[0]) // 2
offset = (real_mid - img_mid) * xm_per_pix

# Unwarp the warped image to overlay lane lines on top of actual image
src = np.float32([[bot_left,top_left,top_right,bot_right]])
dst = np.float32([[dest_bot_left, dest_top_left, dest_top_right, dest_bot_right]])  
m = cv2.getPerspectiveTransform(dst, src)
unwarp_binarywarp = cv2.warpPerspective(warp, m, (imshape[1], imshape[0]))
unwarp_colorwarp  = cv2.warpPerspective(color_warp, m, (imshape[1], imshape[0]))

# Read the undistorted image and overlay lane
undist[unwarp_binarywarp == 255] = (0,0,255)
result = cv2.addWeighted(undist, 1, unwarp_colorwarp, 0.3, 0)

# Put text of curvature radiust and vehicle offset
cv2.putText(result,"left radius = " + str(int(left_curve)) + " m",(100,500), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),3)
cv2.putText(result,"right radius = " + str(int(right_curve)) + " m",(800, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),3)
cv2.putText(result, "vehicle offset = " + str(round(offset,2)) + " m", (550, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),3)  

# Result
temp.left_fit = left_fit
temp.right_fit = right_fit

[ 3.16872075e-04 -5.68997361e-01  5.23123574e+02]
[  -22.02690227   805.85889935 -6559.84634863]


In [543]:
class Line():
    def __init__(self):
        self.hi = True